In [1]:
import pandas as pd
import seaborn as sns
import pandasql as ps
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("data/data.csv")

/Users/peterjmyers/Work/TestProject/venv/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (37,62) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df = df[["X", "Y", "collision_date", "collision_time", "day_of_week", "intersection", "weather_1", "location_type", "collision_severity", "number_killed", "number_injured", "party_count", "primary_coll_factor", "pcf_viol_category", "hit_and_run", "type_of_collision", "road_surface", "road_cond_1", "lighting", "control_device", "pedestrian_accident", "bicycle_accident", "motorcycle_accident", "truck_accident", "alcohol_involved", "count_ped_killed", "count_ped_injured", "count_bicyclist_killed", "count_bicyclist_injured"]]


Fix missing Values

In [4]:
df['location_type']=df['location_type'].fillna('N')

df['pedestrian_accident']=df['pedestrian_accident'].fillna('N')

df['bicycle_accident']=df['bicycle_accident'].fillna('N')

df['motorcycle_accident']=df['motorcycle_accident'].fillna('N')

df['truck_accident']=df['truck_accident'].fillna('N')
df['alcohol_involved']=df['alcohol_involved'].fillna('N')

Drop Duplicates

In [5]:
rows_before = df.shape[0]

df = df.drop_duplicates()

"""rows before: {}, rows after: {}, duplicate rows: {}""". \
format(rows_before, df.shape[0], rows_before - df.shape[0])

'rows before: 171533, rows after: 171522, duplicate rows: 11'

Drop Rows without Geo Location

In [6]:
ps.sqldf("select ROUND(X, 0), ROUND(Y, 0), count(*) from df group by 1,2 order by 1 desc,2 desc", locals())

ps.sqldf("select * from df where ROUND(X, 0) = 0 or ROUND(Y, 0) = 0", locals())

rows_before = df.shape[0]
df = ps.sqldf("select * from df where ROUND(X, 0) != 0 and ROUND(Y, 0) != 0", locals())

"""rows before: {}, rows after: {}, rows dropped: {}""". \
format(rows_before, df.shape[0], rows_before - df.shape[0])

'rows before: 171522, rows after: 169515, rows dropped: 2007'

Get the Grid

In [7]:
df_1 = ps.sqldf("""
SELECT
1 AS sector,
-118.66 AS left,
-118.50 AS right,
34.32 AS up,
34.14 AS down
UNION ALL
SELECT
2 AS sector,
-118.50 AS left,
-118.34 AS right,
34.32 AS up,
34.14 AS down         
UNION ALL
SELECT
3 AS sector,
-118.36 AS left,
-118.14 AS right,
34.14 AS up,
34.05 AS down
UNION ALL
SELECT
4 AS sector,
-118.50 AS left,
-118.36 AS right,
34.14 AS up,
33.91 AS down 
UNION ALL
SELECT
5 AS sector,
-118.36 AS left,
-118.28 AS right,
34.05 AS up,
33.915 AS down
UNION ALL
SELECT
6 AS sector,
-118.28 AS left,
-118.13 AS right,
34.05 AS up,
33.915 AS down
UNION ALL
SELECT
7 AS sector,
-118.325 AS left,
-118.238 AS right,
33.915 AS up,
33.7 AS down

         """, locals())


location_lookup_df = ps.sqldf("""

SELECT
z2.*,
new_left + (new_right - new_left)/2 AS center_x,
new_down + (new_up - new_down)/2 AS center_y
FROM
    (SELECT
      sector,
      (delta_x*row1) + left AS new_left,
      (delta_x*(row1+1)) + left AS new_right,
      (delta_y*row2) + down AS new_down,
      (delta_y*(row2+1)) + down AS new_up,
      x.*,
      y.*,
      z.*

    FROM
        (SELECT
          sector,
          (right - left) / 10 delta_x,
          (up - down) / 10 delta_y,
          left,
          down,
          right,
          up
        FROM df_1
        ) x
    CROSS JOIN
        (
        SELECT 0 as row1 UNION ALL
        SELECT 1 as row1 UNION ALL
        SELECT 2 as row1 UNION ALL
        SELECT 3 as row1 UNION ALL
        SELECT 4 as row1 UNION ALL
        SELECT 5 as row1 UNION ALL
        SELECT 6 as row1 UNION ALL
        SELECT 7 as row1 UNION ALL
        SELECT 8 as row1 UNION ALL
        SELECT 9 as row1
        ) y 
        CROSS JOIN
        (
        SELECT 0 as row2 UNION ALL
        SELECT 1 as row2 UNION ALL
        SELECT 2 as row2 UNION ALL
        SELECT 3 as row2 UNION ALL
        SELECT 4 as row2 UNION ALL
        SELECT 5 as row2 UNION ALL
        SELECT 6 as row2 UNION ALL
        SELECT 7 as row2 UNION ALL
        SELECT 8 as row2 UNION ALL
        SELECT 9 as row2
        ) z
    ) z2
""", locals())
location_lookup_df['index1'] = location_lookup_df.index


Load datasets

In [8]:
dfA = pd.read_csv("data/2010_Census_Populations_by_Zip_Code.csv")
dfB = pd.read_csv("data/Intersections.csv")
dfC = pd.read_csv("data/Regulatory_Signs.csv")
dfD = pd.read_csv("data/Street_Lights.csv")
dfE = pd.read_csv("data/Stop_and_Yield_Signs.csv")
# dfF = pd.read_csv("data/Walkability_Index_Score_2012.csv")

In [9]:
df = ps.sqldf("""
select 
a.*, b.index1, sector
from df a
INNER JOIN
location_lookup_df b
 ON a.X >= new_left AND a.X < new_right
AND a.Y >= new_down AND a.Y < new_up
""", locals())

In [10]:
from uszipcode import ZipcodeSearchEngine
search = ZipcodeSearchEngine()
# zipcode = search.by_zipcode(90001)
res = search.by_city(city="Los Angeles", returns=0)
search.export_to_csv(res, "result.csv")
dfAnew = pd.read_csv("data/result.csv")
dfA.columns = dfA.columns.str.strip().str.lower().str.replace(' ', '_')
dfAcombined = ps.sqldf("""
SELECT Population, Density, Wealthy, HouseOfUnits, Latitude, Longitude, median_age
    from dfAnew a
    LEFT JOIN
    dfA b ON
    a.Zipcode = b.zip_code
""", locals())
population_intermediate_df = ps.sqldf("""
select 
a.*,
b.*,
(((center_x - Longitude)*(center_x - Longitude)) + ((center_y - Latitude)*(center_y - Latitude))) AS distance
from location_lookup_df a
CROSS JOIN
dfAcombined b
""", locals())
population_final_df = ps.sqldf("""
select 
a.index1, Population, Density, Wealthy, HouseOfUnits, median_age
from population_intermediate_df a
INNER JOIN
(select index1, min(distance) AS min_distance from population_intermediate_df group by 1) b
ON a.index1 = b.index1 and
   a.distance = b.min_distance
group by 1
order by 1
""", locals())

In [11]:
intersections_df = ps.sqldf("""
select
index1, count(*) AS intersections
FROM
    (select index1, LAT, LON
    from dfB a
    INNER JOIN
    location_lookup_df b
     ON a.LON >= new_left AND a.LON < new_right
    AND a.LAT >= new_down AND a.LAT < new_up
    ) x
group by 1 
order by 1
""", locals())

street_lights_df = ps.sqldf("""
SELECT
  index1,
  SUM(total_watts) AS total_watts,
  SUM(total_lightbulbs) AS total_lightbulbs
  
FROM
(select index1, 
CASE WHEN TRIM(IFNULL(LAMPA,'')) = '' THEN 0 ELSE SUBSTR(LAMPA, 1, instr(LAMPA,"W")) END +
CASE WHEN TRIM(IFNULL(LAMPB,'')) = '' THEN 0 ELSE SUBSTR(LAMPB, 1, instr(LAMPB,"W")) END +
CASE WHEN TRIM(IFNULL(LAMPC,'')) = '' THEN 0 ELSE SUBSTR(LAMPC, 1, instr(LAMPC,"W")) END +
CASE WHEN TRIM(IFNULL(LAMPD,'')) = '' THEN 0 ELSE SUBSTR(LAMPD, 1, instr(LAMPD,"W")) END +
CASE WHEN TRIM(IFNULL(LAMPE,'')) = '' THEN 0 ELSE SUBSTR(LAMPE, 1, instr(LAMPE,"W")) END +
CASE WHEN TRIM(IFNULL(LAMPF,'')) = '' THEN 0 ELSE SUBSTR(LAMPF, 1, instr(LAMPF,"W")) END AS total_watts,
CASE WHEN TRIM(IFNULL(LAMPA,'')) = '' THEN 0 ELSE 1 END +
CASE WHEN TRIM(IFNULL(LAMPB,'')) = '' THEN 0 ELSE 1 END +
CASE WHEN TRIM(IFNULL(LAMPC,'')) = '' THEN 0 ELSE 1 END +
CASE WHEN TRIM(IFNULL(LAMPD,'')) = '' THEN 0 ELSE 1 END +
CASE WHEN TRIM(IFNULL(LAMPE,'')) = '' THEN 0 ELSE 1 END +
CASE WHEN TRIM(IFNULL(LAMPF,'')) = '' THEN 0 ELSE 1 END AS total_lightbulbs
from dfD a
INNER JOIN
location_lookup_df b
 ON a.X >= new_left AND a.X < new_right
AND a.Y >= new_down AND a.Y < new_up
) x
GROUP BY 1
ORDER BY 1
""", locals())

regulatory_signs_df = ps.sqldf("""
SELECT
  index1,
  SUM(one_way_sign) AS one_way_signs,
  SUM(speed_limit_sign) AS speed_limit_signs,
  SUM(no_u_turn_sign) AS no_u_turn_signs
FROM
(select index1,
CASE WHEN lower(tooltip) like '%one way%' THEN 1 ELSE 0 END AS one_way_sign,
CASE WHEN lower(tooltip) like '%speed%' THEN 1 ELSE 0 END AS speed_limit_sign,
CASE WHEN lower(tooltip) like '%no u turn%' THEN 1 ELSE 0 END AS no_u_turn_sign
from dfC a
INNER JOIN
location_lookup_df b
 ON a.X >= new_left AND a.X < new_right
AND a.Y >= new_down AND a.Y < new_up
) x
GROUP BY 1
ORDER BY 1
""", locals())

stop_and_yield_signs_df = ps.sqldf("""
SELECT
  index1,
  SUM(stop_sign) AS stop_sign,
  SUM(yield_sign) AS yield_sign
FROM
(select index1,
    CASE WHEN lower(tooltip) like '%stop%' THEN 1 ELSE 0 END AS stop_sign,
    CASE WHEN lower(tooltip) like '%yield%' THEN 1 ELSE 0 END AS yield_sign
    from dfE a
    INNER JOIN
    location_lookup_df b
     ON a.X >= new_left AND a.X < new_right
    AND a.Y >= new_down AND a.Y < new_up
    ) x
GROUP BY 1
ORDER BY 1
""", locals())

In [12]:
final_granular_df = ps.sqldf("""
SELECT
*
FROM
df a
LEFT JOIN population_final_df b on b.index1 = a.index1
LEFT JOIN intersections_df c on c.index1 = a.index1
LEFT JOIN street_lights_df d on d.index1 = a.index1
LEFT JOIN regulatory_signs_df e on e.index1 = a.index1
LEFT JOIN stop_and_yield_signs_df f on f.index1 = a.index1
""", locals())

In [13]:
final_granular_df.shape

(166850, 49)

In [14]:
final_granular_df['intersection']=final_granular_df['intersection'].apply(lambda l: "Unknown" if l == '-' else l)
final_granular_df['weather_1']=final_granular_df['weather_1'].apply(lambda l: "Unknown" if l == '-' else l)
final_granular_df['primary_coll_factor']=final_granular_df['primary_coll_factor'].apply(lambda l: "Unknown" if l == '-' else l)
final_granular_df['pcf_viol_category']=final_granular_df['pcf_viol_category'].apply(lambda l: "Unknown" if l == '-' else l)
final_granular_df['pcf_viol_category']=final_granular_df['pcf_viol_category'].apply(lambda l: "Unknown" if l == '- ' else l)
final_granular_df['type_of_collision']=final_granular_df['type_of_collision'].apply(lambda l: "Unknown" if l == '-' else l)
final_granular_df['road_surface']=final_granular_df['road_surface'].apply(lambda l: "Unknown" if l == '-' else l)
final_granular_df['road_cond_1']=final_granular_df['road_cond_1'].apply(lambda l: "Unknown" if l == '-' else l)
final_granular_df['lighting']=final_granular_df['lighting'].apply(lambda l: "Unknown" if l == '-' else l)
final_granular_df['control_device']=final_granular_df['control_device'].apply(lambda l: "Unknown" if l == '-' else l)




In [15]:
columns = final_granular_df.columns
columns

for column in columns:
    if column != "collision_date" and column != "index1" and final_granular_df[column].dtype == "O":
        print(column + " " + str(final_granular_df[column].unique()))

intersection ['Y' 'N' 'Unknown']
weather_1 ['A' 'C' 'B' 'E' 'Unknown' 'F' 'G' 'D']
location_type ['N' 'I' 'H' 'R']
primary_coll_factor ['A' 'D' 'Unknown' 'C' 'B' 'E']
pcf_viol_category ['09' '01' '03' '00' '12' '04' '05' '18' '07' '08' '10' '11' '06' '21' '17'
 '22' 'Unknown' '02' '13' '16' '15' '14' '24']
hit_and_run ['N' 'F' 'M']
type_of_collision ['D' 'A' 'C' 'B' 'G' 'E' 'F' 'Unknown' 'H']
road_surface ['A' 'B' 'Unknown' 'D' 'C']
road_cond_1 ['H' 'A' 'D' 'Unknown' 'B' 'G' 'C' 'E' 'F']
lighting ['A' 'C' 'B' 'D' 'E' 'Unknown']
control_device ['A' 'D' 'B' 'Unknown' 'C']
pedestrian_accident ['N' 'Y']
bicycle_accident ['N' 'Y']
motorcycle_accident ['N' 'Y']
truck_accident ['N' 'Y']
alcohol_involved ['N' 'Y']


In [16]:
final_granular_df = ps.sqldf("""
SELECT
  CASE WHEN collision_time >= 0 and collision_time < 300 THEN 1 else 0 END is_12AM_3AM,
  CASE WHEN collision_time >= 300 and collision_time < 600 THEN 1 else 0 END is_3AM_6AM,
  CASE WHEN collision_time >= 600 and collision_time < 900 THEN 1 else 0 END is_6AM_9AM,
  CASE WHEN collision_time >= 900 and collision_time < 1200 THEN 1 else 0 END is_9AM_12PM,
  CASE WHEN collision_time >= 1200 and collision_time < 1500 THEN 1 else 0 END is_12PM_3PM,
  CASE WHEN collision_time >= 1500 and collision_time < 1800 THEN 1 else 0 END is_3PM_6PM,
  CASE WHEN collision_time >= 1800 and collision_time < 2100 THEN 1 else 0 END is_6PM_9PM,
  CASE WHEN collision_time >= 2100 and collision_time < 2400 THEN 1 else 0 END is_9PM_12AM,
  CASE WHEN day_of_week = 1 THEN 1 ELSE 0 END AS is_monday,
  CASE WHEN day_of_week = 2 THEN 1 ELSE 0 END AS is_tuesday,
  CASE WHEN day_of_week = 3 THEN 1 ELSE 0 END AS is_wednesday,
  CASE WHEN day_of_week = 4 THEN 1 ELSE 0 END AS is_thursday,
  CASE WHEN day_of_week = 5 THEN 1 ELSE 0 END AS is_friday,
  CASE WHEN day_of_week = 6 THEN 1 ELSE 0 END AS is_saturday,
  CASE WHEN day_of_week = 7 THEN 1 ELSE 0 END AS is_sunday,
  x.*
FROM final_granular_df x

""", locals())

In [17]:
final_granular_df.to_csv("a.csv")

In [18]:
df_testfinal2 = pd.get_dummies(final_granular_df, drop_first=True, columns = [
'pedestrian_accident',
'bicycle_accident',
'motorcycle_accident',
'truck_accident',
'alcohol_involved'])

df_testfinal2 = pd.get_dummies(final_granular_df, columns = [
'intersection',
'weather_1',
'location_type',
'primary_coll_factor',
'pcf_viol_category',
'hit_and_run',
'type_of_collision',
'road_surface',
'road_cond_1',
'lighting',
'control_device'
])

In [19]:
df_testfinal2.shape

(166850, 129)

In [20]:
df_testfinal2.head()

,is_12AM_3AM,is_3AM_6AM,is_6AM_9AM,is_9AM_12PM,is_12PM_3PM,is_3PM_6PM,is_6PM_9PM,is_9PM_12AM,is_monday,is_tuesday,...,lighting_B,lighting_C,lighting_D,lighting_E,lighting_Unknown,control_device_A,control_device_B,control_device_C,control_device_D,control_device_Unknown
0,0,0,0,0,0,1,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
1,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,0,0,0,0,0,0,0,1,0,0,...,0,1,0,0,0,1,0,0,0,0


Causes for accidents in general
- then pedestrial, then bicycle, then severe accidents, 


random
- % accidents by time and weekday

In [22]:
df_testfinal2.to_csv("df_testfinal2.csv")


# Aggregated Dataset
- You can skip the code above and start here if you have this dataset

In [41]:
df_testfinal2 = pd.read_csv("df_testfinal2.csv")

In [42]:
df_testfinal2.columns

Index(['Unnamed: 0', 'is_12AM_3AM', 'is_3AM_6AM', 'is_6AM_9AM', 'is_9AM_12PM',
       'is_12PM_3PM', 'is_3PM_6PM', 'is_6PM_9PM', 'is_9PM_12AM', 'is_monday',
       ...
       'lighting_B', 'lighting_C', 'lighting_D', 'lighting_E',
       'lighting_Unknown', 'control_device_A', 'control_device_B',
       'control_device_C', 'control_device_D', 'control_device_Unknown'],
      dtype='object', length=130)

In [43]:
final_aggregated_df = ps.sqldf("""
SELECT
Sector, 
collision_date,
COUNT(*) AS collisions,
SUM(is_12AM_3AM) AS is_12AM_3AM,
SUM(is_3AM_6AM) AS is_3AM_6AM,
SUM(is_6AM_9AM) AS is_6AM_9AM,
SUM(is_9AM_12PM) AS is_9AM_12PM,
SUM(is_12PM_3PM) AS is_12PM_3PM,
SUM(is_3PM_6PM) AS is_3PM_6PM,
SUM(is_6PM_9PM) AS is_6PM_9PM,
SUM(is_9PM_12AM) AS is_9PM_12AM,
MAX(is_monday) AS is_monday,
MAX(is_tuesday) AS is_tuesday,
MAX(is_wednesday) AS is_wednesday,
MAX(is_thursday) AS is_thursday,
MAX(is_friday) AS is_friday,
MAX(is_saturday) AS is_saturday,
MAX(is_sunday) AS is_sunday,
AVG(Density) AS avg_density_per_collision, 
AVG(Wealthy) AS avg_wealth_per_collision,
AVG(intersections) AS avg_intersections_per_collision,
AVG(total_watts) AS avg_watts_per_collision,
AVG(total_lightbulbs) AS avg_lightbulbs_per_collision,
AVG(one_way_signs) AS avg_one_way_signs_per_collision,
AVG(speed_limit_signs) AS avg_speed_limit_signs_per_collision,
AVG(no_u_turn_signs) AS avg_no_u_turn_signs_per_collision,
AVG(stop_sign) AS avg_stop_sign_per_collision,
AVG(yield_sign) AS avg_yield_sign_per_collision,
SUM(CASE WHEN collision_severity IN (1,2) THEN 1 ELSE 0 END) AS high_severity,
SUM(CASE WHEN collision_severity IN (1,2) THEN 0 ELSE 1 END) AS low_severity,
SUM(number_killed) AS number_killed,
SUM(number_injured) AS number_injured,
SUM(pedestrian_accident) AS pedestrian_accident,
SUM(bicycle_accident) AS bicycle_accident,
SUM(motorcycle_accident) AS motorcycle_accident,
SUM(truck_accident) AS truck_accident,
SUM(alcohol_involved) AS alcohol_involved,
SUM(count_ped_killed) AS count_ped_killed,
SUM(count_ped_injured) AS count_ped_injured,
SUM(count_bicyclist_killed) AS count_bicyclist_killed,
SUM(count_bicyclist_injured) AS count_bicyclist_injured,
SUM(intersection_N) AS intersection_N,
SUM(intersection_Unknown) AS intersection_Unknown,
SUM(intersection_Y) AS intersection_Y,
SUM(weather_1_A) AS weather_1_A,
SUM(weather_1_B) AS weather_1_B,
SUM(weather_1_C) AS weather_1_C,
SUM(weather_1_D) AS weather_1_D,
SUM(weather_1_E) AS weather_1_E,
SUM(weather_1_F) AS weather_1_F,
SUM(weather_1_G) AS weather_1_G,
SUM(weather_1_Unknown) AS weather_1_Unknown,
SUM(location_type_H) AS location_type_H,
SUM(location_type_I) AS location_type_I,
SUM(location_type_N) AS location_type_N,
SUM(location_type_R) AS location_type_R,
SUM(primary_coll_factor_A) AS primary_coll_factor_A,
SUM(primary_coll_factor_B) AS primary_coll_factor_B,
SUM(primary_coll_factor_C) AS primary_coll_factor_C,
SUM(primary_coll_factor_D) AS primary_coll_factor_D,
SUM(primary_coll_factor_E) AS primary_coll_factor_E,
SUM(primary_coll_factor_Unknown) AS primary_coll_factor_Unknown,
SUM(pcf_viol_category_00) AS pcf_viol_category_00,
SUM(pcf_viol_category_01) AS pcf_viol_category_01,
SUM(pcf_viol_category_02) AS pcf_viol_category_02,
SUM(pcf_viol_category_03) AS pcf_viol_category_03,
SUM(pcf_viol_category_04) AS pcf_viol_category_04,
SUM(pcf_viol_category_05) AS pcf_viol_category_05,
SUM(pcf_viol_category_06) AS pcf_viol_category_06,
SUM(pcf_viol_category_07) AS pcf_viol_category_07,
SUM(pcf_viol_category_08) AS pcf_viol_category_08,
SUM(pcf_viol_category_09) AS pcf_viol_category_09,
SUM(pcf_viol_category_10) AS pcf_viol_category_10,
SUM(pcf_viol_category_11) AS pcf_viol_category_11,
SUM(pcf_viol_category_12) AS pcf_viol_category_12,
SUM(pcf_viol_category_13) AS pcf_viol_category_13,
SUM(pcf_viol_category_14) AS pcf_viol_category_14,
SUM(pcf_viol_category_15) AS pcf_viol_category_15,
SUM(pcf_viol_category_16) AS pcf_viol_category_16,
SUM(pcf_viol_category_17) AS pcf_viol_category_17,
SUM(pcf_viol_category_18) AS pcf_viol_category_18,
SUM(pcf_viol_category_21) AS pcf_viol_category_21,
SUM(pcf_viol_category_22) AS pcf_viol_category_22,
SUM(pcf_viol_category_24) AS pcf_viol_category_24,
SUM(pcf_viol_category_Unknown) AS pcf_viol_category_Unknown,
SUM(hit_and_run_F) AS hit_and_run_F,
SUM(hit_and_run_M) AS hit_and_run_M,
SUM(hit_and_run_N) AS hit_and_run_N,
SUM(type_of_collision_A) AS type_of_collision_A,
SUM(type_of_collision_B) AS type_of_collision_B,
SUM(type_of_collision_C) AS type_of_collision_C,
SUM(type_of_collision_D) AS type_of_collision_D,
SUM(type_of_collision_E) AS type_of_collision_E,
SUM(type_of_collision_F) AS type_of_collision_F,
SUM(type_of_collision_G) AS type_of_collision_G,
SUM(type_of_collision_H) AS type_of_collision_H,
SUM(type_of_collision_Unknown) AS type_of_collision_Unknown,
SUM(road_surface_A) AS road_surface_A,
SUM(road_surface_B) AS road_surface_B,
SUM(road_surface_C) AS road_surface_C,
SUM(road_surface_D) AS road_surface_D,
SUM(road_surface_Unknown) AS road_surface_Unknown,
SUM(road_cond_1_A) AS road_cond_1_A,
SUM(road_cond_1_B) AS road_cond_1_B,
SUM(road_cond_1_C) AS road_cond_1_C,
SUM(road_cond_1_D) AS road_cond_1_D,
SUM(road_cond_1_E) AS road_cond_1_E,
SUM(road_cond_1_F) AS road_cond_1_F,
SUM(road_cond_1_G) AS road_cond_1_G,
SUM(road_cond_1_H) AS road_cond_1_H,
SUM(road_cond_1_Unknown) AS road_cond_1_Unknown,
SUM(lighting_A) AS lighting_A,
SUM(lighting_B) AS lighting_B,
SUM(lighting_C) AS lighting_C,
SUM(lighting_D) AS lighting_D,
SUM(lighting_E) AS lighting_E,
SUM(lighting_Unknown) AS lighting_Unknown,
SUM(control_device_A) AS control_device_A,
SUM(control_device_B) AS control_device_B,
SUM(control_device_C) AS control_device_C,
SUM(control_device_D) AS control_device_D,
SUM(control_device_Unknown) AS control_device_Unknown
FROM df_testfinal2
GROUP BY 1,2
ORDER BY 1,2

""", locals())
final_aggregated_df.to_csv("final_aggregated_df.csv")

In [2]:
df_pivot = ps.sqldf("""select sector, day_of_week, count(*) count from final_granular_df""")
df2 = df0.df_pivot("sector", "day_of_week", "count")
ax = sns.heatmap(df2)


NameError: name 'ps' is not defined